In [1]:
# Start by unzipping each file in turn if the unzipped version does not exist already

# Once the files have been unzipped read in the data as a dataframe or similar figuring out how to deal with the header properly

# rewrite the data to a better format i.e cleaned data back to 2014 1st of january



Taking zips and extracting the data to the extracted folder

In [2]:
import zipfile
import os

for file in os.listdir('met_eireann_data/zips'):
    if file.endswith('.zip'):
        path_to_zip_file = os.path.join('met_eireann_data/zips', file)
        directory_to_extract_to = os.path.join('met_eireann_data/extracted', file[:-4])  # Remove .zip extension
        if not os.path.exists(directory_to_extract_to):
            os.makedirs(directory_to_extract_to)
        
            with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
                zip_ref.extractall(directory_to_extract_to)

Pipeline to clean data from current format to each row including the station name and all of available weather data

In [13]:
import pandas as pd
dfs = []
for folder in os.listdir('met_eireann_data/extracted'):
    if os.path.isdir(os.path.join('met_eireann_data/extracted', folder)):
        for file in os.listdir(os.path.join('met_eireann_data/extracted', folder)):
            if file.endswith('.csv'):
                skiprows = 0
                with open(f'met_eireann_data/extracted/{folder}/{file}', 'r') as f:
                    for line in f:
                        if line.startswith('Station Name'):
                            station_name = line.split(':')[1].strip()
                        elif line.startswith('Latitude'):
                            Latitude = line.split(',')[0].strip()
                            Longitude = line.split(',')[1].strip()
                        elif line.startswith('Station Height'):
                            Station_Height = line.split(':')[1].strip()
                        elif line.split(',')[0] == 'date':
                            break
                        elif line.split(',')[0] == 'date':
                            break
                        skiprows += 1
                df = pd.read_csv(f'met_eireann_data/extracted/{folder}/{file}', skiprows=skiprows)
                dfs.append((df, station_name, Latitude, Longitude, Station_Height, file))

                # with open(f'met_eireann_data/extracted/{folder}/{file[:-4]}.txt', 'w') as f:
                #     with open(f'met_eireann_data/extracted/{folder}/{file}', 'r') as f2:
                #         for line in f2:
                            
                print(file, station_name, Latitude, Longitude, Station_Height, df.shape)

/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1219256751.py:22: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'met_eireann_data/extracted/{folder}/{file}', skiprows=skiprows)


hly2075.csv FINNER Latitude:54.494 Longitude: -8.243 33 M (239443, 15)
hly1575.csv MALIN HEAD Latitude:55.372 Longitude: -7.339 20 M (16828, 21)


/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1219256751.py:22: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'met_eireann_data/extracted/{folder}/{file}', skiprows=skiprows)


hly275.csv MACE HEAD Latitude:53.326 Longitude: -9.901 21 M (193320, 15)


/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1219256751.py:22: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'met_eireann_data/extracted/{folder}/{file}', skiprows=skiprows)


...

hly1475.csv GURTEEN Latitude:53.035 Longitude: -8.009 75 M (154224, 15)


/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1219256751.py:22: DtypeWarning: Columns (2,4,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'met_eireann_data/extracted/{folder}/{file}', skiprows=skiprows)


hly775.csv SHERKIN ISLAND Latitude:51.476 Longitude: -9.428 21 M (187056, 15)


/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1219256751.py:22: DtypeWarning: Columns (2,4,6,7,8,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'met_eireann_data/extracted/{folder}/{file}', skiprows=skiprows)


hly1075.csv ROCHES POINT Latitude:51.793 Longitude: -8.244 40 M (592127, 15)


In [ ]:
# Check all of the column titles from each file to see if they are consistent
columns = {}
for df_list in dfs:
    df = df_list[0]

    print(df_list[1:],len(df.columns), df.columns)
#     for col in df.columns:
#         if col not in columns:
#             columns[col] = 0
#         columns[col] += 1

# columns


('FINNER', 'Latitude:54.494', 'Longitude: -8.243', '33 M', 'hly2075.csv') 15 Index(['date', 'ind', 'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt',
       'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir'],
      dtype='object')
('MALIN HEAD', 'Latitude:55.372', 'Longitude: -7.339', '20 M', 'hly1575.csv') 21 Index(['date', 'ind', 'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt',
       'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir', 'ww', 'w',
       'sun', 'vis', 'clht', 'clamt'],
      dtype='object')
('MACE HEAD', 'Latitude:53.326', 'Longitude: -9.901', '21 M', 'hly275.csv') 15 Index(['date', 'ind', 'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt',
       'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir'],
      dtype='object')
('MULLINGAR', 'Latitude:53.537', 'Longitude: -7.362', '101 M', 'hly875.csv') 15 Index(['date', 'ind', 'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt',
       'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir'],
      dt

                date  ind  rain  ind.1  temp  ind.2  wetb  dewpt  vappr  rhum  \
0  01-may-1955 01:00    0   0.0      0   8.5      0   7.2    5.5    9.1  83.0   
1  01-may-1955 02:00    0   0.0      0   8.2      0   7.2    6.1    9.3  85.0   
2  01-may-1955 03:00    3   0.0      0   7.3      0   6.6    5.5    9.3  91.0   
3  01-may-1955 04:00    3   0.0      0   7.8      0   7.2    6.1    9.6  91.0   
4  01-may-1955 05:00    0   0.0      0   8.1      0   7.3    6.6    9.7  89.0   

   ...  ind.3  wdsp  ind.4  wddir   ww    w  sun    vis   clht  clamt  
0  ...    0.0  11.0    0.0  130.0  3.0  2.0  0.0  30000   40.0    7.0  
1  ...    0.0  11.0    0.0  120.0  1.0  2.0  0.0  30000  100.0    6.0  
2  ...    0.0   8.0    0.0  120.0  1.0  2.0  0.0  30000  220.0    5.0  
3  ...    0.0  18.0    0.0  130.0  3.0  2.0  0.0  30000   90.0    7.0  
4  ...    0.0   9.0    0.0  110.0  2.0  2.0  0.0  40000   90.0    7.0  

[5 rows x 21 columns]


In [23]:
# Process date column to datetime object and filter to only include data from 2014 onwards
# Add station name at the start and save station metadata in a separate csv file.
# Convert rain, temp, wetb, dewpt, vappr, rhum, msl, wdsp, wddir, sun, vis, clht, clamt to float values if they are columns in the dataframe
# If there are any non float values in these columns (e.g ' ' or '---') convert them to NaN
# Save cleaned data to new csv files in a new folder titled cleaned_data and file name as station_name.csv


from datetime import datetime


for df_list in dfs:
    df, station_name, Latitude, Longitude, Station_Height, file = df_list

    # Process date column to datetime object
    df['date'] = pd.to_datetime(df['date'], format='mixed')
    df = df[df['date'] >= datetime(2014, 1, 1)]

    # clean columns and convert to numeric values 
    columns_to_convert = ['rain', 'temp', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl', 'wdsp', 'wddir', 'sun', 'vis', 'clht', 'clamt']
    for col in columns_to_convert:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    if not os.path.exists('met_eireann_data/cleaned_data'):
        os.makedirs('met_eireann_data/cleaned_data')
    
    # write additional metadata to a separate txt file with the same name as the csv file
    with open(f'met_eireann_data/cleaned_data/{station_name}_metadata.txt', 'w') as f:
        f.write(f'Station Name: {station_name}\n')
        f.write(f'Latitude: {Latitude}\n')
        f.write(f'Longitude: {Longitude}\n')
        f.write(f'Station Height: {Station_Height}\n')

    df.to_csv(f'met_eireann_data/cleaned_data/{station_name}.csv', index=False)

# 21 Index(['date', 'ind', 'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt',
#        'vappr', 'rhum', 'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir', 'ww', 'w',
#        'sun', 'vis', 'clht', 'clamt'],

/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1752708584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')
/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1752708584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')
/var/folders/_p/613_2b4n3p5_5kx0pkkwfw5w0000gs/T/ipykernel_53395/1752708584.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [ ]:
# Load all files from cleaned_data into a single dataframe with an additional column for station name
# Work on exploratory data analysis and visualisation next
# Key visualisations are time series plots of key variables (e.g temperature, wind speed, solar radiation) for each station broken up into years so as to be of a manageable size
